In this code, we organize the data collected from the VNA through the measurement scripts. 

Because the data are not in a format compatible with the Analysis UserFile, a transition script is required to convert them into the proper format.

# Goal

The files collected from the measurement scripts are saved in folders named, for example, "5p932GHz_-25p0dBm_5avgs_201points".

Each folder contains two files:

* A CSV file named "5p932GHz_-25p0dBm_5avgs_201points_001.csv"
* A PNG file named "5p932GHz_-25p0dBm_5avgs_201points_001.png"

Our plan is to extract the CSV file from each folder and rename it to, for example, "Tony_Ta_a_plane_5p932GHz_-25dBm_13mK_.csv".

After renaming, each CSV file will be moved to the target directory, for example, "Resonator_0_5p932GHz".

## Organize and Rename the Measured Files

In [25]:
import os
import re
import glob
import shutil
import numpy as np
import pandas as pd

In [26]:
def grab_and_rename_csvs(
        parent_folder_path,
        filename_template="Tony_Ta_a_plane_{freq}_{power}_13mK_.csv",
        target_folder=None
    ):
    
    """
    Scan subfolders, process CSVs, rename them using template move the renamed files to a target folder.
    """

    # If a target folder is given, ensure it exists
    if target_folder is not None:
        os.makedirs(target_folder, exist_ok=True)

    # List all subdirectories inside parent folder
    subfolders = [f.path for f in os.scandir(parent_folder_path) if f.is_dir()]

    for subfolder_path in subfolders:

        subfolder_name = os.path.basename(subfolder_path)

        # Extract frequency from folder name
        freq_match = re.search(r'(\d+p\d+GHz)', subfolder_name)
        if not freq_match:
            print(f"Skipping {subfolder_name}: Frequency not found.")
            continue
        freq_str = freq_match.group(1)

        # Process CSV files in this subfolder
        csv_files = glob.glob(os.path.join(subfolder_path, "*.csv"))
        for csv_file in csv_files:
            filename = os.path.basename(csv_file)

            # Extract power from filename
            power_match = re.search(r'(-?\d+)p\d+dBm', filename)
            if not power_match:
                print(f"Skipping {filename}: Power not found.")
                continue
            power_str = f"{power_match.group(1)}dBm"

            # Create new filename
            new_filename = filename_template.format(freq=freq_str, power=power_str)

            # Temporary output path (save processed file in parent folder first)
            tmp_filepath = os.path.join(parent_folder_path, new_filename)

            try:
                # --- Detect header row ---
                first_row = pd.read_csv(csv_file, nrows=1, header=None)

                def is_number(x):
                    try:
                        float(x)
                        return True
                    except:
                        return False

                cols_to_check = [c for c in [1, 2, 3] if c in first_row.columns]
                if cols_to_check:
                    numeric_mask = first_row[cols_to_check].map(is_number)
                    first_row_is_header = not numeric_mask.iloc[0].all()
                else:
                    first_row_is_header = True

                skiprows = 1 if first_row_is_header else 0

                # --- Read CSV ---
                df = pd.read_csv(
                    csv_file,
                    skiprows=skiprows,
                    header=None,
                    usecols=[1, 2, 3]
                )

                # --- Phase unit detection ---
                phase_series = df[3].astype(float)
                if (phase_series.abs() <= 180).all():
                    df[3] = np.rad2deg(phase_series)  # Convert from rad to deg

                # --- Save processed CSV ---
                df.to_csv(tmp_filepath, index=False, header=False)

                # --- Move file to target folder if requested ---
                if target_folder is not None:
                    target_path = os.path.join(target_folder, new_filename)

                    # If file exists in target folder, remove to avoid error
                    if os.path.exists(target_path):
                        os.remove(target_path)

                    shutil.move(tmp_filepath, target_path)
                    print(f"✅ Saved: {new_filename}.")

            except Exception as e:
                print(f"❌ Failed to process {filename}: {e}")

In [27]:
# # ----- Function 1 -----
# # Input the path of the parent folder that contains the subfolders, each of which includes the measured CSV files (and PNG files)

# def grab_and_rename_csvs(parent_folder_path):
    
#     # Iterate over every entry (files and folders) inside parent_folder_path, and keep only the entries that are directories (not files).
#     subfolder = [f.path for f in os.scandir(parent_folder_path) if f.is_dir()]

#     for subfolder in subfolder:
        
#         subfolder_name = os.path.basename(subfolder)

#         # Extract frequency from subfolder name, e.g., "5p932GHz_-25p0dBm_5avgs_201points"
#         freq_match = re.search(r'(\d+p\d+GHz)', subfolder_name)
#         if not freq_match:
#             print(f"Skipping {subfolder_name}: Frequency not found in folder name.")
#             continue
#         freq_str = freq_match.group(1)

#         # Process each CSV file in subfolder
#         csv_files = glob.glob(os.path.join(subfolder, "*.csv"))
#         for csv_file in csv_files:
#             filename = os.path.basename(csv_file)

#             # Extract power from filename, e.g., "-20p0dBm" -> "-20dBm"
#             power_match = re.search(r'(-?\d+)p\d+dBm', filename)
#             if not power_match:
#                 print(f"Skipping {filename}: Power not found.")
#                 continue
#             power_str = f"{power_match.group(1)}dBm"

#             # Compose output filename
#             new_filename = f"Tony_Ta_a_plane_{freq_str}_{power_str}_13mK_.csv"
#             new_filepath = os.path.join(parent_folder_path, new_filename)

#             try:
#                 # Read data: skip first row, get columns 1, 2, 3
#                 df = pd.read_csv(csv_file, skiprows=1, header=None, usecols=[1, 2, 3])

#                 # Convert S21 phase from radians to degrees
#                 df[3] = np.rad2deg(df[3])

#                 # Save processed file to main folder
#                 df.to_csv(new_filepath, index=False, header=False)
#                 print(f"✅ Saved: {new_filename}")
#             except Exception as e:
#                 print(f"❌ Failed to process {filename}: {e}")

In [28]:
parent_folder_path = r"D:\cooldown72\Line5\Tony_Ta_a_plane_01\12_01_1523"
target_folder = r"C:\Users\user\Documents\GitHub\Measurements\Cooldown_72_Line5-Tony_Ta_a_plane_01\most_recent_data\Resonator_0_5p932GHz"
grab_and_rename_csvs(
    parent_folder_path=parent_folder_path,
    filename_template="Tony_Ta_a_plane_{freq}_{power}_13mK.csv",
    target_folder=target_folder
)

✅ Saved: Tony_Ta_a_plane_5p932GHz_-25dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-27dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-30dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-32dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-35dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-38dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-40dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-45dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-50dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-52dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-55dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-57dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-60dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-62dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-65dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-67dBm_13mK.csv.
✅ Saved: Tony_Ta_a_plane_5p932GHz_-69dBm_13mK.csv.
